<a href="https://colab.research.google.com/github/MNoichl/tttms_public/blob/main/downloading_biorxiv_fulltexts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary packages

In [ ]:
!pip install xmltodict
!pip install flatten_dict
!pip install boto3

# Load packages and link google drive

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

import re

import tqdm

import tarfile
import glob
import os

from shutil import copyfile, copy, copy2
import shutil
import xmltodict

import gzip
import zipfile
import boto3, configparser



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download BioRxiv-fulltext and images from Amazon-Drive


In [ ]:
def rm_make(dir):
  try:
    shutil.rmtree(dir)
  except:
    pass

  try:
    !mkdir $dir
  except:
    pass

def unzip(source,target):
  with zipfile.ZipFile(source, 'r') as zip_ref:
    zip_ref.extractall(target)



def return_dir_paths(path):
  all_files = []
  for path, subdirs, files in os.walk(path):
      for name in files:
          all_files.append(os.path.join(path, name))
  return all_files

In [ ]:
!rm -rf config.ini
# censored for review before publication

with open('config.ini', 'w') as f:
    f.write("""[DEFAULT]
ACCESS_KEY = ACCESS_KEY
SECRET_KEY =SECRET_KEY""")

In [ ]:
import calendar

query_months = []
for year in [2019,2020]:
  for month in list(calendar.month_name)[1:]:  
    query_months.append(month+ '_' + str(year))
print(query_months)

In [ ]:
configs = configparser.SafeConfigParser()
configs.read('config.ini')


bucket_name = 'biorxiv-src-monthly'
# _PREFIX = 'Current_Content/July_2020'




client = boto3.client('s3', aws_access_key_id=configs['DEFAULT']['ACCESS_KEY'],
                            aws_secret_access_key=configs['DEFAULT']['SECRET_KEY'],
                      )


for this_month in query_months:

  rm_make('big_temp')
  prefix = 'Current_Content/' + this_month

  paginator = client.get_paginator('list_objects_v2')
  pages = paginator.paginate(Bucket=bucket_name,RequestPayer='requester', Prefix=prefix)

  this_months_objects = []
  for page in pages:
      for obj in page['Contents']:
        this_months_objects.append(obj['Key'])
  for this_object in tqdm.tqdm_notebook(this_months_objects):



    if this_object[-4:] == 'meca':
      rm_make('temp_zip')
      file_id = this_object.split('/')[-1]
      
      client.download_file(
        Bucket=bucket_name,
        Key=this_object,  # name of key to download from
        Filename='temp_zip/'+file_id,  # path to file to download to
        ExtraArgs={"RequestPayer": "requester"},
      )
      rm_make('temp')
      unzip('temp_zip/'+file_id, 'temp')

      preprint_folder = 'big_temp/'+file_id.replace('.meca','')
      rm_make(preprint_folder)
      for this_file in return_dir_paths('temp/content'):
        if ('eqn' in this_file) or (this_file[-3:] == 'xml'):
          shutil.copy(this_file, preprint_folder+'/'+this_file.split('/')[-1])

  shutil.make_archive("drive/MyDrive/biorxiv_fulltext_zip_files/"+this_month, 'zip', 'big_temp')


    
      

In [ ]:
this_months_objects

The following code is adapted/taken from a tutorial by Brienna Herold, https://towardsdatascience.com/how-to-bulk-access-arxiv-full-text-preprints-58026e19e8ef

In [ ]:
import boto3, configparser

s3resource = None

def setup():
    """Creates S3 resource & sets configs to enable download."""

    print('Connecting to Amazon S3...')

    # Securely import configs from private config file
    configs = configparser.SafeConfigParser()
    configs.read('config.ini')

    # Create S3 resource & set configs
    global s3resource
    s3resource = boto3.resource(
        's3',  # the AWS resource we want to use
        aws_access_key_id=configs['DEFAULT']['ACCESS_KEY'],
        aws_secret_access_key=configs['DEFAULT']['SECRET_KEY'],
        region_name='us-east-1'  # same region arxiv bucket is in
    )

if __name__ == '__main__':
    """Runs if script is called on command line"""

    # Create S3 resource & set configs
    setup()

In [ ]:
configs = configparser.SafeConfigParser()
configs.read('config.ini')


my_bucket = s3resource.Bucket('biorxiv-src-monthly')
_BUCKET_NAME = 'biorxiv-src-monthly'
_PREFIX = 'Current_Content/July_2020'

client = boto3.client('s3', aws_access_key_id=configs['DEFAULT']['ACCESS_KEY'],
                            aws_secret_access_key=configs['DEFAULT']['SECRET_KEY'],
                      )

# def ListFiles(client):
#     """List files in specific S3 URL"""
#     response = client.list_objects(Bucket=_BUCKET_NAME,RequestPayer='requester', Prefix=_PREFIX)
#     print(response)
#     for content in response.get('Contents', []):
#         yield content.get('Key')

# file_list = ListFiles(client)
# for file in file_list:
#     print('File found: %s' % file)


response = client.list_objects(Bucket=_BUCKET_NAME,RequestPayer='requester', Prefix=_PREFIX)
print(response)

In [ ]:

paginator = client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=_BUCKET_NAME,RequestPayer='requester', Prefix=_PREFIX)

In [ ]:
for page in pages:
    # for obj in page['Contents']:
    print(page['Contents'])
        # print(obj['Size'])

In [ ]:
test_key = 'Current_Content/July_2020/'#340fc728-6cb4-1014-83f1-fe3fe01219f7.meca'

client.download_file(
  Bucket=_BUCKET_NAME,
  Key=test_key,  # name of key to download from
  Filename='test.meca',  # path to file to download to
  ExtraArgs={"RequestPayer": "requester"},
)

In [ ]:
!unzip test.meca

In [ ]:
yimport boto3, configparser, os, botocore

def download_file(key):
    """
    Downloads given filename from source bucket to destination directory.
    Parameters
    ----------
    key : str
        Name of file to download
    """

    # Ensure src directory exists 
    if not os.path.isdir('src'):
        os.makedirs('src')

    # Download file
    print('\nDownloading s3://arxiv/{} to {}...'.format(key, key))

    try:
        s3resource.meta.client.download_file(
            Bucket='arxiv', 
            Key=key,  # name of file to download from
            Filename=key,  # path to file to download to
            ExtraArgs={'RequestPayer':'requester'})
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print('ERROR: ' + key + " does not exist in arxiv bucket")

if __name__ == '__main__':
    """Runs if script is called on command line"""

    # Download manifest file to current directory
    download_file('src/arXiv_src_manifest.xml')

In [ ]:
with open('src/arXiv_src_manifest.xml', 'r') as file:
    data = file.read()
xml_content = xmltodict.parse(data)


In [ ]:
manifest_frame = pd.DataFrame(xml_content['arXivSRC']['file'])

In [ ]:
def rm_make(dir):
  try:
    shutil.rmtree(dir)
  except:
    pass

  try:
    !mkdir $dir
  except:
    pass


def return_dir_paths(path):
  all_files = []
  for path, subdirs, files in os.walk(path):
      for name in files:
          all_files.append(os.path.join(path, name))
  return all_files

In [ ]:
output_directory = 'drive/MyDrive/arxiv_2019_2022_tex_collection_fourth_run'
# rm_make(output_directory)

In [ ]:
all_query_codes = []
for year in tqdm.tqdm_notebook(selected_years):
  for month in tqdm.tqdm_notebook(range(1,13)):
    all_query_codes.append(year[-2:]+str(month).zfill(2))

In [ ]:
# all_query_codes


In [ ]:
for query_code in ['2012']: # all_query_codes[3:]:
  print('Downloading code: ', query_code)
  this_years_queries = manifest_frame[manifest_frame["yymm"] == query_code]
  for ix, row in tqdm.tqdm_notebook(this_years_queries.iterrows()):
    key = row["filename"]
    print('    Downloading file: ', key)
    try:
      rm_make("src")
      s3resource.meta.client.download_file(
        Bucket="arxiv",
        Key=key,  # name of key to download from
        Filename=key,  # path to file to download to
        ExtraArgs={"RequestPayer": "requester"},
      )

      # extract the whole package to temporary directory
      rm_make("temporary_files")
      rm_make("temporary_files_extracted")
      tar = tarfile.open(key, "r:")
      tar.extractall(path="temporary_files")
      tar.close()

      # loop over all extracted files, copying the .tex files to our output directory, into their own folder named after their arxiv id
      all_files = return_dir_paths("temporary_files")
      print('    Extracting file: ', key)
      for this_file in tqdm.tqdm_notebook(all_files):
        this_file_path = this_file.split("/")[-1].replace(".gz", "")
        try:
          if this_file[-2:] == "gz":
            tar = tarfile.open(this_file)
            tar.extractall(
              path="temporary_files_extracted/" + this_file_path
            )
            tar.close()
            for extracted_file in return_dir_paths(
              "temporary_files_extracted/" + this_file_path
            ):
              if extracted_file.endswith(".tex"):
                try:
                  copy(
                    extracted_file,
                    output_directory
                    + "/"
                    + this_file_path
                    + "/"
                    + extracted_file.split("/")[-1],
                  )
                except IOError as io_err:
                  os.makedirs(output_directory + "/" + this_file_path)
                  copy(
                    extracted_file,
                    output_directory
                    + "/"
                    + this_file_path
                    + "/"
                    + extracted_file.split("/")[-1],
                  )

        except (KeyboardInterrupt, SystemExit):
          print("program stopped manually")
          raise
        except:
          # print('file')
          pass

    except botocore.exceptions.ClientError as e:
      if e.response["Error"]["Code"] == "404":
        print("ERROR: " + key + " does not exist in arxiv bucket")


In [ ]:
# /content/drive/MyDrive/arxiv_2019_2022_tex_collection_fourth_run



import shutil
shutil.make_archive('drive/MyDrive/arxiv_2019_2022_tex_collection_fourth_run.zip', 'zip', 'drive/MyDrive/arxiv_2019_2022_tex_collection_fourth_run')

In [ ]:
this_file_path

In [ ]:
extracted_file

In [ ]:
# plt.scatter([float(x[0:4]) for x in manifest_frame.yymm],
#          [float(x) for x in manifest_frame['size']])
# manifest_frame[manifest_frame['yymm'] == '1901']

In [ ]:
# [float(x[0:4]) for x in manifest_frame.timestamp]

In [ ]:
# paginator = s3resource.meta.client.get_paginator('list_objects_v2')

# page_iterator = paginator.paginate(
#   Bucket='arxiv',
#   RequestPayer='requester',
#   Prefix='src/'
# )

# for page in page_iterator:
#   for x in page['Contents']:
#     print(x)